# Model recovery example with stimulus coding for the starting point for evidence accumulation,  $z$
  #### Here, $A_z = z$ and $B_z = 1 - z$

In [1]:
import hddm
from patsy import dmatrix  # for generation of (regression) design matrices
import numpy as np         # for basic matrix operations
from pandas import Series  # to manipulate data-frames generated by hddm

/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [2]:
import sys
sys.stdout = open('ModelRecoveryOutput_test_stim_coding.txt', 'w')

In [3]:
#set n subjects and n trials per condition
n_subjects = 10
trials_per_level = 150 # and per stimulus

In [36]:
#set generative ddm parameters for stimulus A
level1a = {'v':.3, 'a':2, 't':.3,  'z':.5}
level2a = {'v':.4, 'a':2, 't':.3, 'z':.6}
level3a = {'v':.5, 'a':2, 't':.3, 'z':.7}

all_params_stimA = [level1a, level2a, level3a]

In [37]:
#set generative ddm parameters for stimulus B 
#z for stim B is 1-A_z 
level1b = {'v':.3, 'a':2, 't':.3, 'z':(1-level1a['z'])}
level2b = {'v':.4, 'a':2, 't':.3, 'z':(1-level2a['z'])}
level3b = {'v':.5, 'a':2, 't':.3,'z':(1-level3a['z'])}
all_params_stimB = [level1b, level2b, level3b]

In [5]:
#generate the data given the above parameters
data_a, params_a = hddm.generate.gen_rand_data({'level1': level1a,
                                                'level2': level2a,
                                                'level3': level3a},
                                                size=trials_per_level,
                                                subjs=n_subjects)
data_b, params_b = hddm.generate.gen_rand_data({'level1': level1b,
                                                'level2': level2b,
                                                'level3': level3b},
                                                size=trials_per_level,
                                                subjs=n_subjects)


In [22]:
data_a.head(10)

,rt,response,subj_idx,condition,stimulus
0,1.301343,1.0,0,level3,1.0
1,0.749050,0.0,0,level3,1.0
2,0.744004,1.0,0,level3,1.0
3,0.714742,1.0,0,level3,1.0
4,0.391612,1.0,0,level3,1.0
5,1.811101,0.0,0,level3,1.0
6,0.366794,1.0,0,level3,1.0
7,0.670123,1.0,0,level3,1.0
8,0.782597,1.0,0,level3,1.0
9,2.927301,1.0,0,level3,1.0


In [23]:
data_b.head(10)

,rt,response,subj_idx,condition,stimulus
0,0.599324,1.0,0,level3,2.0
1,0.904324,0.0,0,level3,2.0
2,0.935324,1.0,0,level3,2.0
3,0.844324,1.0,0,level3,2.0
4,2.476324,1.0,0,level3,2.0
5,0.734324,0.0,0,level3,2.0
6,0.564324,1.0,0,level3,2.0
7,0.880324,1.0,0,level3,2.0
8,0.774324,1.0,0,level3,2.0
9,1.114324,0.0,0,level3,2.0


In [18]:
#identify the stimuli 
# a = 1 & b = 2
data_a['stimulus'] = Series(np.ones((len(data_a))), index=data_a.index)
data_b['stimulus'] = Series(np.ones((len(data_b)))*2, index=data_a.index)

In [24]:
#merge data_a & data_b
mydata = data_a.append(data_b, ignore_index=True)

In [27]:
#set up the z link function to constrain z from 0:1 
def z_link_func(x, data=mydata):
    stim = (np.asarray(dmatrix('0 + C(s, [[1], [-1]])',
                               {'s': data.stimulus.ix[x.index]}))
    )
    return 1 / (1 + np.exp(-(x * stim)))
#here, stim is 1 & -1 and the result is multiplied appropriately 

In [28]:
#set up the regression model for z 
z_reg = {'model': 'z ~ 1 + C(condition)', 'link_func': z_link_func}

In [29]:
#v does not need the logit function because it's unconstrained. 
#though if simulated that way, it could be useful to do that. 

#if data are response coded, then v would need a link function, but only to transform
#drift-rate values associated with stimuli a & b to - & + values, respectively
v_reg = {'model': 'v ~ 1 + C(condition)', 'link_func': lambda x: x}

In [30]:
#the full regression description
reg_descr = [z_reg, v_reg]

In [60]:
#run the regression model 
m_reg = hddm.HDDMRegressor(mydata, reg_descr, include='z', informative=False)
m_reg.sample(5000, burn=20)

/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/pymc/Node.py:403: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.__name__ = input['__name__']
/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)
/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will 

In [48]:
#compare results to gen. model 
m_reg.gen_stats()

/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)
/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)
/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)
/home/krista/anaconda3/envs/hddm/lib/python3.5/site-packages/numpy/core/fromnumeric.py:224: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return reshape(newshape, order=order)
/home/krista/ana

,mean,std,2.5q,25q,50q,75q,97.5q,mc err
a,2.0809,0.0514583,1.98135,2.04843,2.08015,2.11215,2.18506,0.000415276
a_std,0.15646,0.0468608,0.0922164,0.124061,0.14739,0.178515,0.27254,0.000490712
a_subj.0,1.92207,0.0311667,1.8619,1.90057,1.92184,1.94295,1.98408,0.000362946
a_subj.1,2.06756,0.0314706,2.00641,2.046,2.06742,2.08859,2.12964,0.000301412
a_subj.2,2.15656,0.0316492,2.09612,2.13496,2.15608,2.17757,2.22018,0.000299279
a_subj.3,1.99533,0.0289931,1.93971,1.9754,1.99502,2.01507,2.05228,0.000260944
a_subj.4,2.07984,0.0311598,2.02007,2.0586,2.07949,2.10077,2.14221,0.000299323
a_subj.5,2.27832,0.036078,2.20946,2.25367,2.27788,2.30248,2.35139,0.0003093
a_subj.6,2.14552,0.0317236,2.08426,2.12412,2.14537,2.16663,2.20868,0.000254218
a_subj.7,2.22525,0.0361472,2.15538,2.20056,2.22446,2.24964,2.29764,0.000328594


In [42]:
'gen_param stim A: ', all_params_stimA, 'gen_param stim B: ', all_params_stimB

('gen_param stim A: ',
 [{'a': 2, 't': 0.3, 'v': 0.3, 'z': 0.5},
  {'a': 2, 't': 0.3, 'v': 0.4, 'z': 0.6},
  {'a': 2, 't': 0.3, 'v': 0.5, 'z': 0.7}],
 'gen_param stim B: ',
 [{'a': 2, 't': 0.3, 'v': 0.3, 'z': 0.5},
  {'a': 2, 't': 0.3, 'v': 0.4, 'z': 0.4},
  {'a': 2, 't': 0.3, 'v': 0.5, 'z': 0.30000000000000004}])

In [61]:
#level1_z 
1/(1+np.exp(-(0.121011))) 

#level 2_z 
1/(1+np.exp(-(0.121011 + 0.15))) 

#level 3_z 
1/(1+np.exp(-(0.121011 + 0.52))) 


0.65498196276784049